In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Reshape, Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Dense, Lambda, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import numpy as np
import pickle
import os, cv2
import pandas as pd

Using TensorFlow backend.


Converted first 2000 training images into array and appended them into a list X.
Since we renamed the training images from 1.jpg to 14000.jpg we can use a loop to retrieve them.

In [2]:
X = []

for i in range(1,2001):
    myFile = "/content/drive/My Drive/newtraining/training_images/" + str(i) + ".png"
    if i%100==0:
        print(myFile)
    image = cv2.imread(myFile)
    X.append (image)

/content/drive/My Drive/newtraining/training_images/0.png
/content/drive/My Drive/newtraining/training_images/100.png
/content/drive/My Drive/newtraining/training_images/200.png
/content/drive/My Drive/newtraining/training_images/300.png
/content/drive/My Drive/newtraining/training_images/400.png
/content/drive/My Drive/newtraining/training_images/500.png
/content/drive/My Drive/newtraining/training_images/600.png
/content/drive/My Drive/newtraining/training_images/700.png
/content/drive/My Drive/newtraining/training_images/800.png
/content/drive/My Drive/newtraining/training_images/900.png
/content/drive/My Drive/newtraining/training_images/1000.png
/content/drive/My Drive/newtraining/training_images/1100.png
/content/drive/My Drive/newtraining/training_images/1200.png
/content/drive/My Drive/newtraining/training_images/1300.png
/content/drive/My Drive/newtraining/training_images/1400.png
/content/drive/My Drive/newtraining/training_images/1500.png
/content/drive/My Drive/newtraining/

Saved this list into the drive using Pickle

In [0]:
with open("/content/drive/My Drive/newtraining/1401k_batch.txt", "wb") as fp:
    pickle.dump(X, fp)

In [0]:
del X

In [0]:
with open("/content/drive/My Drive/newtraining/6k_batch.txt", "rb") as fp:   # Unpickling
    X = pickle.load(fp)

In [0]:
X = np.array(X)

In [0]:
X.shape

(2000, 480, 640, 3)

In [ ]:
X = X.astype(np.float32)
X.dtype

Normalised each channel to make mean=0 and std deviation = 1

In [ ]:
mean2 = X[:,:,:,2].mean() 
std2=X[:,:,:,2].std()
mean1 = X[:,:,:,1].mean() 
std1=X[:,:,:,1].std()
mean = X[:,:,:,0].mean() 
std=X[:,:,:,0].std()
X[:,:,:,0] = (X[:,:,:,0] - mean)/std
X[:,:,:,1] = (X[:,:,:,1] - mean1)/std1
X[:,:,:,2] = (X[:,:,:,2] - mean2)/std2

Loaded the training.csv file

In [0]:
boundary = pd.read_csv("/content/drive/My Drive/newtraining/training.csv")
boundary.head(5)

,image_name,x1,x2,y1,y2
0,JPEG_20160706_121146_1000145715002.png,115,495,143,325
1,JPEG_20161119_174038_1000690577600.png,23,457,61,409
2,147444927651111470309333776-Roadster-Men-Casua...,37,601,13,470
3,147772332675720161028_161611.png,27,602,162,385
4,1473315333651DeeplearnS11638.png,60,586,174,325


In [0]:
df = boundary
del df["image_name"]

Slicing of the csv file to get first 2000 boundary boxes

In [0]:
df = df.iloc[0:2000 , :]
df.shape

(2000, 4)

In [0]:
y = df.values
y.shape

(2000, 4)

In [0]:
y = y.astype(np.float32)
y.dtype

dtype('float32')

In [0]:
    model = Sequential()

    model.add(Conv2D(24, (5, 5), padding='same', input_shape=X.shape[1:], name='block1_conv1'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))

    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

    model.add(Conv2D(36, (5, 5), padding='same', name='block2_conv2'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

    model.add(Conv2D(48, (5, 5), padding='same', name='block3_conv1'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))

    model.add(Conv2D(64, (3, 3), padding='same', name='block3_conv3'))
    model.add(BatchNormalization())
    model.add(Activation('elu'))


    model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

    model.add(Conv2D(64, (3, 3), padding='valid', name='block4_conv1'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))

    model.add(Flatten())

    model.add(Dense(10))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))

    model.add(Dense(8, name='fc2'))
    model.add(BatchNormalization()) 
    model.add(Activation('elu'))
    model.add(Dropout(rate = 0.4))

    model.add(Dense(4))


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 480, 640, 24)      1824      
_________________________________________________________________
batch_normalization_21 (Batc (None, 480, 640, 24)      96        
_________________________________________________________________
activation_21 (Activation)   (None, 480, 640, 24)      0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 240, 320, 24)      0         
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 240, 320, 36)      21636     
_________________________________________________________________
batch_normalization_22 (Batc (None, 240, 320, 36)      144       
_________________________________________________________________
activation_22 (Activation)   (None, 240, 320, 36)      0         
__________

In [0]:
model.compile(loss='mean_absolute_error', optimizer=Adam(lr=0.1))

In [ ]:
model.fit(X, y, epochs=40)

Saved the model weights after training of first batch

In [0]:
model.save("/content/drive/My Drive/newtraining/model2kbatch.h5")

In [0]:
del model